In [9]:
import json
import uuid

import boto3
from boto3.dynamodb.conditions import Key

dyn_resource = boto3.resource("dynamodb")

TABLE_NAME = "ipynb-test-table"

In [2]:
# Delete table if it already exists
try:
    table = dyn_resource.Table(TABLE_NAME)
    table.delete()
    table.wait_until_not_exists()
    print(f"Deleted table {TABLE_NAME}")
except Exception as e:
    print(f"Table didn't exist yet, you're good:\n{e}")

Deleted table ipynb-test-table


In [3]:
def create_ddb_table(table_name=TABLE_NAME):
    params = {
        "TableName": table_name,
        "KeySchema": [
            {"AttributeName": "partition_key", "KeyType": "HASH"},
            # {"AttributeName": "sort_key", "KeyType": "RANGE"},
        ],
        "AttributeDefinitions": [
            {"AttributeName": "partition_key", "AttributeType": "S"},  # S is string
            # {"AttributeName": "sort_key", "AttributeType": "S"},
        ],
        "ProvisionedThroughput": {"ReadCapacityUnits": 10, "WriteCapacityUnits": 10},
    }
    table = dyn_resource.create_table(**params)
    print(f"Creating {table_name}...")
    table.wait_until_exists()
    print("Done.")
    return

In [4]:
create_ddb_table()

Creating ipynb-test-table...


Done.


In [5]:
# Grab a pointer to the table
table = dyn_resource.Table(TABLE_NAME)

In [6]:
# Write some data to table
# Use random UUID as partition key
# Create a field called "some_data" with a string in it
this_uuid = str(uuid.uuid4())
this_data = "This is some data!"
table.put_item(
    Item={
        "partition_key": this_uuid,
        "some_data": this_data,
    }
)

{'ResponseMetadata': {'RequestId': '2295PE3EIJIDKNHS603H71I2NFVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 14 Apr 2024 16:28:13 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '2295PE3EIJIDKNHS603H71I2NFVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [7]:
# Get the data back from the table based on partition_key
kazu: dict = table.get_item(Key={"partition_key": this_uuid})
print(kazu["Item"]["some_data"])

This is some data!


In [8]:
# Test query functionality
# Query on primary key (you can additionally query on sort key)
key_condition_expression = Key("partition_key").eq(this_uuid)
qkazu: dict = table.query(KeyConditionExpression=key_condition_expression)
print(f"Found {len(qkazu['Items'])} values from query.")
print(qkazu["Items"][0])

Found 1 values from query.
{'some_data': 'This is some data!', 'partition_key': '56b9155d-e84c-48e8-974b-d7391c051a7b'}


In [10]:
# Let's make sure we can store json data (by dump to string)
j_uuid = str(uuid.uuid4())
j_data = {"foo": {"bar": 4}}
table.put_item(
    Item={
        "partition_key": j_uuid,
        "some_data": json.dumps(j_data),
    }
)

{'ResponseMetadata': {'RequestId': '7N634SI3MGL3A9HCH47TAGGNRRVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 14 Apr 2024 16:30:48 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '7N634SI3MGL3A9HCH47TAGGNRRVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [11]:
# Get the json data back from the table based on partition_key
kazu: dict = table.get_item(Key={"partition_key": j_uuid})
retrieved_j_data = json.loads(kazu["Item"]["some_data"])
print(retrieved_j_data)

{'foo': {'bar': 4}}


In [15]:
# How do I add another row to existing partition?
def update_entry(primary_key, new_col_name, new_col_value):
    # Define the key of the item you want to update
    key = {"partition_key": primary_key}

    # Define the attribute(s) you want to add or update
    update_expression = "SET #new_attr = :new_value"
    expression_attribute_names = {"#new_attr": new_col_name}
    expression_attribute_values = {":new_value": new_col_value}

    # Update the item
    table.update_item(
        Key=key,
        UpdateExpression=update_expression,
        ExpressionAttributeNames=expression_attribute_names,
        ExpressionAttributeValues=expression_attribute_values,
    )

In [16]:
update_entry(this_uuid, "NEW DATA!", "This is some new data!")

In [17]:
kazu: dict = table.get_item(Key={"partition_key": this_uuid})
print(kazu["Item"])

{'some_data': 'This is some data!', 'NEW DATA!': 'This is some new data!', 'partition_key': '56b9155d-e84c-48e8-974b-d7391c051a7b'}


In [18]:
# And overwriting existing columns?
update_entry(this_uuid, "NEW DATA!", "Does this overwrite?")

In [19]:
kazu: dict = table.get_item(Key={"partition_key": this_uuid})
print(kazu["Item"])

{'some_data': 'This is some data!', 'NEW DATA!': 'Does this overwrite?', 'partition_key': '56b9155d-e84c-48e8-974b-d7391c051a7b'}


In [5]:
# For my use case, the first lambda will create a unique id
# that is the partition key. It will create a dynamodb entry
# with the parent media URI as data.
# Future lambdas will all only know about that partition key.

dynamodb.Table(name='foo')